In [4]:
import pandas as pd
import numpy as np
import datetime as dt 
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# чтение сгенерированного файла
df = pd.read_csv('data/pervozki_with_regions.csv')

In [6]:
# удаление колонок, которые не дают важных результатов в процессе исследования
df.drop(['kommentariy_perevozki',
         'id_voditelya',
         'fio_voditelya',
         'gos_nomer_avto',
         'punkt_zagruzki',
         'punkt_razgruzki',
         'nalichie_dokumentov',
         'kategorija_perevozki',
         'otklonenie_ot_plana_chas',
         'kompaniya_perevozchik'], axis=1, inplace=True)

In [7]:
# работа над колонкой отправителя/получателя, где данные контрагентов были переименованы в определенные группы
def groupping_companies(name):
    name = name.split()
    if name[0]=='ЗАО' or name[0]=='ОАО' or name[0]=='АО' or name[0]=='РАО' or name[0]=='ООО':
        return 'коммерческая компания'
    elif name[0]=='ИП':
        return 'индивидуальный предприниматель'
    elif name[0]=='НПО':
        return 'некоммерческая организация'
    else:
        return 'коммерческая компания'
    

# применение функции к колонкам грузоотправителя / грузополучателя 
df['gruzopoluchatel'] = df['gruzopoluchatel'].apply(groupping_companies)
df['gruzootpravitel'] = df['gruzootpravitel'].apply(groupping_companies)


# изменение данных грузоотправителя на "интернет - магазин" по перевозкам с типом грузов "ТНП"
df.loc[df["tip_gruza"] == "ТНП", "gruzootpravitel"] = "интернет-магазин"

In [8]:
# деление данных колонки даты для извлечения даты, исключая времени
def splitting_date(name):
    return name.split()[0]

# применение функции к колонкам
df['data_viezda'] = df['data_viezda'].apply(splitting_date)
df['planovaya_data_pribytiya'] = df['planovaya_data_pribytiya'].apply(splitting_date)
df['data_sozdaniya_zakaza'] = df['data_sozdaniya_zakaza'].apply(splitting_date)
df['fakticheskaya_data_pribytiya'] = df['fakticheskaya_data_pribytiya'].apply(splitting_date)

In [9]:
# перевод из строкового типа данных на тип данных "даты/время"
df['data_sozdaniya_zakaza'] = pd.to_datetime(df['data_sozdaniya_zakaza'])
df['data_viezda'] = pd.to_datetime(df['data_viezda'])
df['planovaya_data_pribytiya'] = pd.to_datetime(df['planovaya_data_pribytiya'])
df['fakticheskaya_data_pribytiya'] = pd.to_datetime(df['fakticheskaya_data_pribytiya'])


In [10]:
# обработка колонок статус:
# "в пути" (у грузов в статусе "в пути" нет фактической даты прибытия)  
# "запланирована" (у грузов в статусе "запланирована" нет фактической даты прибытия, даты выезда)
# "отменена" (грузы данного статуса удалены, так как не несут какой-либо информации и грузы не были отданы под отвественность транспортной компании)
df.loc[df['status_perevozki'] == "В пути", "fakticheskaya_data_pribytiya"] = pd.NaT
df.loc[df['status_perevozki'] == "Запланирована", ["fakticheskaya_data_pribytiya", "data_viezda"]] = pd.NaT
df.loc[df['status_perevozki'] == "Отменена", ["fakticheskaya_data_pribytiya", "data_viezda","planovaya_data_pribytiya"]] = pd.NaT


In [11]:
# создание новой колонки срока доставки для отслеживания сроков перевозки грузов
df['srok_dostavki'] = df['planovaya_data_pribytiya'] - df['data_viezda']
df['otklonenie_ot_plana_dney'] = df['fakticheskaya_data_pribytiya'] - df['planovaya_data_pribytiya']


# замена пустых значений на ноль
df['srok_dostavki'] = df['srok_dostavki'].fillna(pd.Timedelta(days=0))
df['otklonenie_ot_plana_dney'] = df['otklonenie_ot_plana_dney'].fillna(pd.Timedelta(days=0))


# удаление строк с отрицательным значением (возможные тех.ошибки записи датасета)
df.drop(df[df["srok_dostavki"] < pd.Timedelta(0)].index, inplace=True)
df.drop(df[df["otklonenie_ot_plana_dney"] < pd.Timedelta(0)].index, inplace=True)


In [12]:
# преобразование в тип целые числа для выведения верных данных на стримлит
df['srok_dostavki'] = df['srok_dostavki'].apply(lambda x: str(x)[0])
df['srok_dostavki'] = df['srok_dostavki'].astype(int)
df['otklonenie_ot_plana_dney'] = df['srok_dostavki'].apply(lambda x: str(x)[0])
df['otklonenie_ot_plana_dney'] = df['srok_dostavki'].astype(int)

In [13]:
# округление стоимостей
df['tarif_summa'] = df['tarif_summa'].apply(lambda x: round(x))
df['sebestoimost_perevozki'] = df['sebestoimost_perevozki'].apply(lambda x: round(x))
df['pribyl'] = df['pribyl'].apply(lambda x: round(x))

In [14]:
# определение категориальных значений по дням задержки и образование новой колонки
def identification_otklonenie_ot_plana_dney(day):
    if day <=2:
        return 'Незначительный'
    elif 2 < day <=4:
        return 'Тревожный'
    elif day >=5:
        return 'Нарушенный'
    else:
        return 'Не нарушен'
    
df['identification_otklonenie_ot_plana_dney'] = df['otklonenie_ot_plana_dney'].apply(identification_otklonenie_ot_plana_dney)

In [15]:
# сохранение удобного порядка колонок для удобного восприятия информации
df = df[['data_sozdaniya_zakaza',
         'status_perevozki',
         'gruzootpravitel',
         'gruzopoluchatel',
         'tip_gruza',
         'gorod_zagruzki',
         'gorod_razgruzki',
         'rasstoyanie_km',
         'marka_model_avto', 
         'tip_kuzova',
         'gruzopodemnost_tonn', 
         'data_viezda',
         'planovaya_data_pribytiya',
         'fakticheskaya_data_pribytiya',
         'identification_otklonenie_ot_plana_dney',
         'srok_dostavki',
         'tonnazh_gruza_tonn',
         'obem_gruza_m3',
         'kolichestvo_pozitsiy',
         'tarif_summa',
         'sebestoimost_perevozki',
         'pribyl'
         ]]

In [19]:
(df['sebestoimost_perevozki']/df['tarif_summa'] * 100).mean()

np.float64(75.16323501195163)

In [ ]:
# экспорт обработанного датасета
# df.to_csv('data/perevozki_processing.csv', index=False)